# Visualisation interactive d'objets mathématiques avec *ipywidgets* et construction d'applications

### Odile Bénassy

### Projet OpenDreamKit, LRI & Université Paris Sud

### Sage Days, 19 février 2020



* **visual** representations: plots, ..
* interactivity: interactive **widgets**
* how to build **small applications** with only a few lines of code

Suppose we want to study how Taylor series can approximate the $sinus$ function.

$$sin: x \mapsto sin(x)$$

In [ ]:
s = taylor(sin(x), x, 0, 5)

In [ ]:
plot([x, s,sin], xmin=-5, xmax=5, ymin=-2, ymax=2, color=["gray", "blue","red"])

In [ ]:
@interact(degree=(1,25,2))
def f(degree=1):
    s = taylor(sin(x), x, 0, degree)
    display(plot([x, s,sin], xmin=-5, xmax=5, ymin=-2, ymax=2, color=["gray", "blue","red"]))
    

`@interact` permet d'obtenir très rapidement une petite application interactive, avec déjà des possibilités de paramétrage riches.

Cependant, il n'est pas possible de l'utiliser ensuite comme **brique de base** pour une application.

Voici un exemple d'une telle application. Nous allons mélanger une liste en plusieurs étapes.

In [ ]:
import random

def melange(l, duration):
    for t in range(duration):
        i = random.randint(0, len(l)-1)
        j = random.randint(0, len(l)-1)
        l[i], l[j] = l[j], l[i] 
        
l = list(range(10)) ; l

In [ ]:
melange(l, 10); l

Mieux comprendre l'algorithme? Traçons l'exécution visuellement:

In [ ]:
import copy

def melange(l, duration):
    history = [copy.copy(l)]
    for t in range(duration):
        i = random.randint(0, len(l)-1)
        j = random.randint(0, len(l)-1)
        l[i], l[j] = l[j], l[i]
        history.append(copy.copy(l))
    return history

l = list(range(10))
history = melange(l, 100)

## Notre petite application

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from bqplot import Bars, LinearScale, Figure
from ipywidgets import IntSlider, Button, HTML, VBox

l = history[0]
bars = Bars(x=range(len(l)), y = l, scales={'x': LinearScale(), 'y': LinearScale()})
w1 = Figure(marks=[bars])
slider = IntSlider(0, 0, 99) # equiv. IntSlider(min=0, max=99)
def update(change):
    w1.marks[0].y = history[change["new"]]
slider.observe(update, names='value')
VBox((slider, w1))

In [ ]:
current_index2 = 0
bars = Bars(x=range(len(l)), y = history[current_index2], scales={'x': LinearScale(), 'y': LinearScale()})
w2 = Figure(marks=[bars])
button =  Button(description="Next")
label1 = HTML()
def button_clicked(b):
    global current_index2
    current_index2 += 1
    w2.marks[0].y = history[current_index2]
    label1.value = '<p style="font-size:24px; color:tomato">Step #%d</p>' % current_index2
button.on_click(button_clicked)
VBox((button, w2, label1))

In [ ]:
from ipywidgets import HTML
from traitlets import Int, observe
class Indicator(HTML):
    step= Int()
    @observe('step')
    def step_changed(self, change):
        self.value = '<p style="font-size:24px; color:tomato">Step #%d</p>' % self.step 

In [ ]:
current_index = 0
bars = Bars(x=range(len(l)), y = history[current_index], scales={'x': LinearScale(), 'y': LinearScale()})
w = Figure(marks=[bars])
button =  Button(description="Next")
label2 = Indicator()
def button_clicked(b):
    global current_index
    current_index += 1
    w.marks[0].y = history[current_index]
    label2.step = int(current_index)
button.on_click(button_clicked)
VBox((button, w, label2))

## Composition de _widgets_ 

In [ ]:
from ipywidgets import *
b = Button(description="My button")
t = Textarea()
s = Select(options=("1", "2", "3"))
d = Dropdown(options=("1", "2", "3"))
#HBox((b,t,s))

In [ ]:
#VBox((b,d,t))

* IntText, ToggleButton, FloatRangeSlider ..
* HBox, VBox, GridBox, Accordion, Tabs .. 

Liste des widgets `ipywidgets` disponibles

* https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

* 'ipywidgets list'

In [ ]:
#GridBox((b,b,b,d,d,d,t,t,t), layout=Layout(grid_template_columns="repeat(3, 300px)"))

# Composition et héritage

## Sage Combinat Widgets

In [ ]:
#from sage_combinat_widgets import GridViewWidget
#t = StandardTableaux(15).random_element()
#GridViewWidget(t)

In [ ]:
%display unicode_art
Sym = SymmetricFunctions(QQ['t']);
s = Sym.s()
s[3,1].coproduct()

In [ ]:
from sage_combinat_widgets.grid_view_widget import PartitionGridViewWidget
@interact
def f(p1 = PartitionGridViewWidget(Partition([2,1]))): 
      return s[p1].coproduct()

## Sage Explorer

In [ ]:
from sage_explorer import explore
#sp = SkewPartition()
#sin?
#explore(sin)
#dir(sin)

In [ ]:
from sage_explorer import explore
%display unicode_art
p = Partition([7, 4, 2, 1])
explore(p)

## Francy

In [ ]:
import networkx
E = FiniteSetMaps(4)
f1 = E([0,0,2,3])
f2 = E([0,1,1,3])
f3 = E([0,1,2,2])
H = E.submonoid([f1, f2, f3])
g = H.cayley_graph(side='twosided', simple=True)
G = networkx.DiGraph()
G.add_edges_from([(e[0], e[1]) for e in g.edges()])

In [ ]:
from francy_widget import FrancyWidget
llvs = g.level_sets()
levels = {}
for lvs in llvs:
    for n in lvs:
        levels[n] = llvs.index(lvs)

def node_options(n):
    options = {}
    if n.is_idempotent():
        options['type'] = 'diamond'       
    else:
        options['type'] = 'circle'
    options['layer'] = levels[n]
    options['modal_menus'] = [{
        'title': 'cardinality',
        'funcname': 'cardinality',
        'is_method': True
    }]
    return options

fw = FrancyWidget(G, 
                  title="NDPF4", height=600, weight=0, graphType="directed",
                  node_options=node_options)
fw

## Emmy Noether's descent

Modules utilisés dans cette présentation :
    * ipywidgets
    * bqplot
    * random
    * copy
    * traitlets
    * sage_combinat_widgets
    * sage_explorer
    * networkx
    * francy
    * RISE

## Questions ?

    odile.benassy@u-psud.fr
    
`zerline` sur Github